In [1]:
import subprocess
import os
os.environ['CURL_CA_BUNDLE'] = ''

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

## 配置及导入必要的包

In [2]:
from diffusers import StableDiffusionPipeline
import torch
from datetime import datetime
import uuid

In [3]:
today = datetime.now().strftime('%m%d')
today

'1105'

In [4]:
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
torch_dtype = torch.float16 if 'cuda' in device else torch.float32
torch_dtype = torch.float32
torch_dtype

torch.float32

## Model & Config & Forward

不同精度下，模型加载和运行所需显存量：

- torch.float16
    - load：3588MiB-8
    - forward: 5896MiB-8
- torch.float32
    - load: 6268MiB-8
    - forward: 11064MiB-8

In [6]:
pipe = StableDiffusionPipeline.from_pretrained('runwayml/stable-diffusion-v1-5', 
                                               torch_dtype=torch_dtype).to(device)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
pipe.vae.config

FrozenDict([('in_channels', 3),
            ('out_channels', 3),
            ('down_block_types',
             ['DownEncoderBlock2D',
              'DownEncoderBlock2D',
              'DownEncoderBlock2D',
              'DownEncoderBlock2D']),
            ('up_block_types',
             ['UpDecoderBlock2D',
              'UpDecoderBlock2D',
              'UpDecoderBlock2D',
              'UpDecoderBlock2D']),
            ('block_out_channels', [128, 256, 512, 512]),
            ('layers_per_block', 2),
            ('act_fn', 'silu'),
            ('latent_channels', 4),
            ('norm_num_groups', 32),
            ('sample_size', 512),
            ('scaling_factor', 0.18215),
            ('shift_factor', None),
            ('latents_mean', None),
            ('latents_std', None),
            ('force_upcast', True),
            ('use_quant_conv', True),
            ('use_post_quant_conv', True),
            ('mid_block_add_attention', True),
            ('_use_default_values',
     

In [8]:
pipe.unet.config

FrozenDict([('sample_size', 64),
            ('in_channels', 4),
            ('out_channels', 4),
            ('center_input_sample', False),
            ('flip_sin_to_cos', True),
            ('freq_shift', 0),
            ('down_block_types',
             ['CrossAttnDownBlock2D',
              'CrossAttnDownBlock2D',
              'CrossAttnDownBlock2D',
              'DownBlock2D']),
            ('mid_block_type', 'UNetMidBlock2DCrossAttn'),
            ('up_block_types',
             ['UpBlock2D',
              'CrossAttnUpBlock2D',
              'CrossAttnUpBlock2D',
              'CrossAttnUpBlock2D']),
            ('only_cross_attention', False),
            ('block_out_channels', [320, 640, 1280, 1280]),
            ('layers_per_block', 2),
            ('downsample_padding', 1),
            ('mid_block_scale_factor', 1),
            ('dropout', 0.0),
            ('act_fn', 'silu'),
            ('norm_num_groups', 32),
            ('norm_eps', 1e-05),
            ('cross_attenti

In [9]:
a_prompt = 'best quality, extremely detailed'
n_prompt = 'longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'

In [10]:
prompt = 'a photo of an astronaut riding a horse on mars'
prompt = prompt + ', ' + a_prompt
prompt

'a photo of an astronaut riding a horse on mars, best quality, extremely detailed'

In [11]:
img = pipe(prompt, negative_prompt=n_prompt).images[0]

  0%|          | 0/50 [00:00<?, ?it/s]

In [12]:
img.save(f'./results/image-{today}-{str(uuid.uuid4())[:8]}.png')